#import

In [1]:
import numpy as np
import pandas as pd
import librosa

In [2]:
# 資料判斷
df_csv = pd.read_csv("Training Dataset/training datalist.csv")
print("資料資訊")
df_csv.info()

資料資訊
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ID                         1000 non-null   object 
 1   Sex                        1000 non-null   int64  
 2   Age                        1000 non-null   int64  
 3   Disease category           1000 non-null   int64  
 4   Narrow pitch range         1000 non-null   int64  
 5   Decreased volume           1000 non-null   int64  
 6   Fatigue                    1000 non-null   int64  
 7   Dryness                    1000 non-null   int64  
 8   Lumping                    1000 non-null   int64  
 9   heartburn                  1000 non-null   int64  
 10  Choking                    1000 non-null   int64  
 11  Eye dryness                1000 non-null   int64  
 12  PND                        1000 non-null   int64  
 13  Smoking                    1000 non-null   i

In [5]:
# 挑選出要訓練的Disease category 1、2、3
df_csv = df_csv.loc[df_csv['Disease category'].isin([1, 2, 3]), ['ID', 'Disease category']]

# 在dataframe中加入要訓練的音檔路徑
df_csv['wav_path'] = df_csv['ID'].map("./Training Dataset/training_voice_data{}.wav".format)

print("Disease category in source_df :",df_csv['Disease category'].unique())
print("source_df :\n", df_csv)

Disease category in source_df : [1 2 3]
source_df :
           ID  Disease category  \
0    1202f15                 1   
1    0600ve0                 2   
2    1001o7l                 2   
3    1201c1t                 1   
4    0402jvt                 1   
..       ...               ...   
994  0T022un                 3   
995  0G00ftn                 3   
997  0202p64                 3   
998  12021au                 2   
999  04027it                 2   

                                              wav_path  
0    ./Training Dataset/training_voice_data1202f15.wav  
1    ./Training Dataset/training_voice_data0600ve0.wav  
2    ./Training Dataset/training_voice_data1001o7l.wav  
3    ./Training Dataset/training_voice_data1201c1t.wav  
4    ./Training Dataset/training_voice_data0402jvt.wav  
..                                                 ...  
994  ./Training Dataset/training_voice_data0T022un.wav  
995  ./Training Dataset/training_voice_data0G00ftn.wav  
997  ./Training Dataset/t